- Data Preprocessing
- Comparing means
- T tests
- Analysis of variance
- Nonparametric statistical tests


## Part 1: Preparing data for Statistcal Analysis

Original Source: https://github.com/divya-gh/Spotify_Music_Analysis

Data is taken from [Kaggle](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks) which has been authored by Yamac Eren Ay and  was collected using Spotify Web API.

In [ ]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load Data


In [ ]:
file_path = "Resources/data.csv"
spotify_data_df = pd.read_csv(file_path)

## Display DataFrame



In [ ]:
spotify_data_df.head()

## Display total music releases by artists over 100 years

In [ ]:
rows = spotify_data_df.shape[0]
columns = spotify_data_df.shape[1]
total_before = pd.DataFrame({
                     " Total Rows": [rows],
                     " Total Columns": [columns]
                    })
total_before

The dataset contains a total of 1,74,389 records by artists till date and has 19 audio characteristics.

## Understand and Explore Audio Characteristics

Get a sense of data you collected, such as artisits, song names, popularity, etc and their types.


In [ ]:
spotify_data_df.columns

In [ ]:
spotify_data_df.dtypes

## Feature Description: Explain how they are measured and what they mean.
### Primary:
    •	- id : Song unique ID

### Numerical:
    •	- acousticness : A confidence measure from 0.0 to 1.0 of whether the track is acoustic; 1-High , 0 -Low
    •	- danceability: How suitable a track is for dancing based on a combination of musical elements; 0-Least, 1-Most
    •	- energy: Measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy;0-Low,1-High
    •	- duration_ms : Duration of song in milliseconds ranging from 200k to 300k
    •	- instrumentalness :Predicts whether a track contains no vocals(ex: oohh and aahh) ;1-No vocals, 0-vocal
    •	- valence : Musical positiveness conveyed by a track.
            0-negative(sad, depressed, angry) ,
            1-positive (happy, cheerful, euphoric)
    •	- popularity: Based on number of plays and downloads ; 0-Less Popular, 100-Very Popular
    •	- tempo : Overall estimated tempo of a track in beats per minutes(BPM);50-Low , 100-High
    •	- liveness: Detects the presence of an audience in the recording ; 0-No , 1 -Yes
    •	- loudness: Overall relative loudness of tracks in decibels (dB). -60 low ,0-High
    •	- speechiness: presence of spoken words in a track.
              Values >0.66-entirely of spoken words.
              Values between 0.33 and 0.66 describe tracks that may contain both music and speech,like rap music.
              Values < 0.33 mostly music and other non-speech-like tracks.
    •	- year : The year music was recorded and release (1921 to 2020)

### Dummy:
    •	- mode (0 = Minor, 1 = Major)
    •	- explicit (0 = No explicit content, 1 = Explicit content)

### Categorical:
    •	- key : All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1
    •	- artists :Song Artist
    •	- release_date : Date when album released
    •	- name : Song Name


###  Interesting insights from data exploration:
    -	This data is simply a sample of tracks released in those 100 years and not a complete set.
    -	According to Spotify developers site, the popularity is calculated by an algorithm and is based on the
         total number of downloads and plays the track has had and how recent those plays are. While this is accurate for
         newer tracks, could be a bias on older tracks.
    -   Valance descibes the mood of the singer. It would be interesting to know that it was measured based on
         loudness. Louder the song, happier and cheerful it is.
    -   Not all the features are measured in the range 0-1 . ex: To plot the trending of loudness with other features
         over time, we would have to normalize the units to fit the scale.

## Preprocessing and Feature Selection

- Perform basic data cleanup.
- Select features that fit to the stats analysis interests.[link text](https://)
- Here we keep them all.

In [ ]:
spotify_data_df.head(5)

Data needs a cleanup:
- Remove special charecters from the artists column.
- Parse data types and
- Drop Null values.

In [ ]:
# remove special characters from the column 'artist' using lstrip , rstrip  and str.replace functions (aka "[]")
spotify_data_df_clean_artists= spotify_data_df.copy()
spotify_data_df_clean_artists["artists"]=spotify_data_df_clean_artists["artists"].map(lambda x: x.lstrip("['").rstrip("']")).astype(str)


In [ ]:
spotify_data_df_clean_artists.head(5)

In [ ]:
# remove single and double quotes
spotify_data_df_clean_artists["artists"]=spotify_data_df_clean_artists["artists"].str.replace("'","").str.replace('"',"")

In [ ]:
spotify_data_df_clean_artists

In [ ]:
# drop Null values in the df if any
spotify_data_df_clean =spotify_data_df_clean_artists.dropna(how='any')
spotify_data_df_clean.shape[0]

**Note** : None of the rows were dropped so dataframe has no Null values

All the features in interest have expected dtypes hence, no columns need any datatype conversion.

# Part 2: Descriptive Statistics -  Initial Analysis after Cleanup

In [ ]:
spotify_data_df_clean.describe()

### Basic statistical analysis
                       
- No. of releases: 174389
- Start year: 1920             
- End Year: 2021   
             
Note:  From the table, we can see that there are records with loudness greater than zero. According to spotify developers site, range is between -60 to 0 db .This could be an error so we need to remove these records.

In [ ]:
spotify_data_df_clean = spotify_data_df_clean[spotify_data_df_clean["loudness"]<0]
loudness_after = pd.DataFrame({
                        "max_loudness":[spotify_data_df_clean["loudness"].max()],
                        "min_loudness":[spotify_data_df_clean["loudness"].min()]
                        })
loudness_after

In [ ]:
spotify_data_df_clean.head(3)

In [ ]:
ideal_order = ['id','name','year', 'artists', 'duration_ms',  'release_date',
               'energy', 'acousticness', 'danceability',
               'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
               'mode', 'popularity', 'speechiness', 'tempo',
               'valence' ]
spotify_df = spotify_data_df_clean[ideal_order]


# Random sample of 1000 rows
spotify_df = spotify_df.sample(n=10000, random_state=42)
spotify_df.head(5)

In [ ]:
rows = spotify_df.shape[0]
columns = spotify_df.shape[1]
total_after = pd.DataFrame({
                     " Total Rows": [rows],
                     " Total Columns": [columns]
                    })
total_after

# **Part 3: T tests**

## Independent Samples t-test


### Example:

Compare two *independent* groups: explicit=0 vs explicit=1.

### Hypothesis:

- H0: Explicit and non-explicit songs have the same mean energy.
- H1: There is a significant difference in energy between explicit and non-explicit songs.


## Check Normal Distribution

choices of measures: parametric / non-parametric.

In [ ]:
import seaborn as sns
import scipy.stats as stats

# group the data
group1 = spotify_df[spotify_df['explicit'] == 0]['energy']
group2 = spotify_df[spotify_df['explicit'] == 1]['energy']

# === 1. Histogram with KDE ===
plt.figure(figsize=(10, 5))
sns.histplot(group1, kde=True, color='blue', label='Non-Explicit', stat='density')
sns.histplot(group2, kde=True, color='orange', label='Explicit', stat='density')
plt.title('Histogram of Energy by Explicit Flag')
plt.xlabel('Energy')
plt.ylabel('Density')
plt.legend()
plt.show()

# === 2. QQ Plots ===
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# QQ Plot for Non-Explicit
stats.probplot(group1, dist="norm", plot=axes[0])
axes[0].set_title("QQ Plot - Non-Explicit")

# QQ Plot for Explicit
stats.probplot(group2, dist="norm", plot=axes[1])
axes[1].set_title("QQ Plot - Explicit")

plt.tight_layout()
plt.show()


In [ ]:
# import t-test library
from scipy.stats import ttest_ind

group1 = spotify_df[spotify_df['explicit'] == 0]['energy']
group2 = spotify_df[spotify_df['explicit'] == 1]['energy']

print(f"Explicit=0: {len(group1)}, Explicit=1: {len(group2)}")

# check mean difference
mean_explicit = group2.mean()
mean_non_explicit = group1.mean()

print(f"Mean energy (explicit): {mean_explicit:.4f}")
print(f"Mean energy (non-explicit): {mean_non_explicit:.4f}")
print(f"Difference: {mean_explicit - mean_non_explicit:.4f}")

# equal_var -- unequal variance
# setting equal_var to False uses Welch's t-test
t_stat, p_value = ttest_ind(group1, group2, equal_var=False)

print(f"t-statistic: {t_stat:.4f}, p-value: {p_value:.8f}")


### Plot Mean Difference

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x='explicit', y='energy', data=spotify_df)
plt.title('Energy Distribution by Explicit Flag')
plt.xticks([0, 1], ['Non-Explicit', 'Explicit'])
plt.show()


### Degree of Freedom

Since groups are very different in size and may have different variances, we use Welch's t-test, which adjusts the degrees of freedom with the Welch–Satterthwaite formula. This gives a more accurate p-value when assumptions of equal variance are violated.


In [ ]:
import numpy as np

# Get sample sizes
n1, n2 = len(group1), len(group2)

# Get sample variances
s1_sq = np.var(group1, ddof=1)
s2_sq = np.var(group2, ddof=1)

# Welch–Satterthwaite equation
numerator = (s1_sq/n1 + s2_sq/n2)**2
denominator = ((s1_sq/n1)**2)/(n1 - 1) + ((s2_sq/n2)**2)/(n2 - 1)
df_welch = numerator / denominator

print(f"Welch's degrees of freedom: {df_welch:.2f}")

#### T-tests Interpretation

- There is a statistically significant difference in energy between explicit and non-explicit songs.
- The negative t-statistic suggests that explicit songs have higher energy (if you did explicit=1 minus explicit=0).
- We reject the null hypothesis (p < 0.05).

**Note** Welch's t-test was correctly used because of unequal group sizes and unknown variances.

## Paired Samples t-test

### Example: paired features from the same observation on songs.


### Hypothesis:
- H0: The danceability of a song and its valence are equal on average.
- H1: The average danceability and valence differ within the same song.


In [ ]:
# import paired-samples t-test
from scipy.stats import ttest_rel

# check mean difference
mean_dance = spotify_df['danceability'].mean()
mean_valence = spotify_df['valence'].mean()

print(f"Mean Danceability: {mean_dance:.4f}")
print(f"Mean Valence: {mean_valence:.4f}")
print(f"Mean Difference: {mean_dance - mean_valence:.4f}")

t_stat, p_value = ttest_rel(spotify_df['danceability'], spotify_df['valence'])

print(f"t-statistic: {t_stat:.4f}, p-value: {p_value:.4f}")


### Plot Mean Difference



In [ ]:

# distribution of differences
diff = spotify_df['danceability'] - spotify_df['valence']

sns.histplot(diff, kde=True)
plt.axvline(0, color='red', linestyle='--')
plt.title('Distribution of Danceability - Valence')
plt.xlabel('Difference')
plt.ylabel('Count')
plt.show()


#### Paired-samples t-test Interpretation

- We reject the null hypothesis: The average danceability and valence scores of songs are not equal.
- Since the t-statistic is positive, it suggests: Danceability is significantly greater than valence, on average.



## One-tailed t-test

### Hypothesis (One-tailed):
- H0: Popularity of songs from year <= 2010 is greater than or equal to popularity of songs from year > 2010.
- H1: Songs from after 2010 are more popular.

In [ ]:
pre_2010 = spotify_df[spotify_df['year'] <= 2010]['popularity']
post_2010 = spotify_df[spotify_df['year'] > 2010]['popularity']

t_stat, p_value = ttest_ind(post_2010, pre_2010, equal_var=False)

# one-tailed: divide p-value by 2 and check direction
if t_stat > 0:
    p_one_tailed = p_value / 2
else:
    p_one_tailed = 1 - (p_value / 2)

print(f"One-tailed p-value: {p_one_tailed:.4f}")


### Plot Difference

In [ ]:
spotify_df['era'] = spotify_df['year'].apply(lambda x: 'Pre-2010' if x <= 2010 else 'Post-2010')

sns.boxplot(x='era', y='popularity', data=spotify_df)
plt.title('Popularity: Pre-2010 vs Post-2010')
plt.xlabel('Era')
plt.ylabel('Popularity')
plt.show()

In [ ]:
means = spotify_df.groupby('era')['popularity'].mean()
stds = spotify_df.groupby('era')['popularity'].std()
counts = spotify_df.groupby('era')['popularity'].count()
sems = stds / np.sqrt(counts)  # standard error of mean

means.plot(kind='bar', yerr=sems, capsize=4, color=['gray', 'green'])
plt.title('Mean Popularity by Era (± SEM)')
plt.ylabel('Popularity')
plt.show()

### Interpretation

A one-tailed t-test is appropriate when the hypothesis has a direction — whether post-2010 songs are more popular than pre-2010.

The extremely low p-value shows the result is significant, and the plots show a clear shift in popularity.

# **Part 4: ANOVA**


### Example: Testing 3 or more groups.

### Hypothesis:
- H0: The mean tempo is the same across all keys (0–11).
- H1: At least one key group has a significantly different mean tempo.


In [ ]:
from scipy.stats import f_oneway

# group by 'key' (12 groups)
groups = [group['tempo'].values for _, group in spotify_df.groupby('key')]

f_stat, p_value = f_oneway(*groups)

print(f"F-statistic: {f_stat:.4f}, P-value: {p_value:.4f}")


#### Interpretation

p < 0.001

There is a statistically significant difference in the mean value of the dependent variable across at least one of the groups in the independent variable.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# treat the variable key as a categorical factor (not numeric) -- variation between groups.
model = ols('tempo ~ C(key)', data=spotify_df).fit()

# Get full ANOVA table
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table['mean_sq'] = anova_table['sum_sq'] / anova_table['df']
anova_table['eta_sq'] = anova_table['sum_sq'] / anova_table['sum_sq'].sum()

print(anova_table)


> The F-value of 33.18 tells us the between-group variance (due to key) is much greater than the within-group variance (error). The p-value < 0.001 confirms this is statistically significant. But notice the eta squared: key explains ~0.28% of the total variance in tempo — so it's significant, but not the main factor.



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x='key', y='tempo', data=spotify_df)
plt.title('Tempo Distribution by Musical Key')
# charles if you want you can chage the labels.
plt.xlabel('Key')
plt.ylabel('Tempo')
plt.show()

## Factorial ANOVA (Two way)


> Does a song's tempo vary depending on both its musical key and mode (major/minor)?

Key = 12 levels (categorical)
Mode = 0 (minor), 1 (major)

- Test main effects of key and mode
- Test interaction effect between key × mode

In [ ]:
# model: tempo ~ key + mode + key:mode
model = ols('tempo ~ C(key) + C(mode) + C(key):C(mode)', data=spotify_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# add mean square and effect size
anova_table['mean_sq'] = anova_table['sum_sq'] / anova_table['df']
anova_table['eta_sq'] = anova_table['sum_sq'] / anova_table['sum_sq'].sum()

print(anova_table)


In [ ]:

sns.pointplot(x='key', y='tempo', hue='mode', data=spotify_df, dodge=True, markers=['o', 's'], capsize=0.1)
plt.title("Interaction: Key × Mode on Tempo")
plt.ylabel("Tempo")
plt.show()


## Repeated Measures ANOVA

### Example:
> Within each song, are there significant differences between the following audio features?

- danceability
- energy
- valence

This is a within-subject design because each song has all 3 measures.

In [ ]:
# TODO.

df_long = spotify_df[['id', 'danceability', 'energy', 'valence']].melt(id_vars='id',
                                                               value_vars=['danceability', 'energy', 'valence'],
                                                               var_name='feature',
                                                               value_name='score')

df_long_clean = (
    df_long
    .groupby(['id', 'feature'], as_index=False)['score']
    .mean()
)

# from statsmodels.stats.anova import AnovaRM

# anova_rm = AnovaRM(df_long_clean, depvar='score', subject='id', within=['feature'])
# results = anova_rm.fit()

# print(results)

# Part 5: Nonparametric statistical tests

Test whether there is any relationship in categorical variables.

## Chi-Square Test of Independence

### Hypothesis:

> Is there a relationship between explicit content and mode (major/minor)?

- explicit = 0 or 1
- mode = 0 (minor) or 1 (major)

This data is usually presented in tables of counts.

In [ ]:
from scipy.stats import chi2_contingency

# let's make them to a 2*2 table
contingency_table = pd.crosstab(spotify_df['explicit'], spotify_df['mode'])

print(contingency_table)

In [ ]:
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2:.4f}")
print(f"Degrees of freedom: {dof}")
print(f"P-value: {p:.4f}")

#### Interpretation

-  Since p < 0.05, we reject the null hypothesis which explicit content and musical mode are not independent. 
- There is a significant relationship between them.

In [ ]:
data = {
    'Group': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'],
    'Answer': ['Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
}
df = pd.DataFrame(data)
contingency_table = pd.crosstab(df['Group'], df['Answer'])
print(contingency_table)
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-square statistic: {chi2:.4f}")
print(f"Degrees of freedom: {dof}")
print(f"P-value: {p:.4f}")